In [26]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

In [27]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

Indexing - Document Ingestion

In [28]:
import os
from dotenv import load_dotenv
load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY") 

In [47]:
video_id = "Gfr50f6ZBvo" # only the ID, not full URL
try:
    # If you don’t care which language, this returns the “best” one
    transcript_list = YouTubeTranscriptApi().fetch(video_id, languages=["en"])

    # Flatten it to plain text
    transcript = " ".join(chunk.text for chunk in transcript_list)

except TranscriptsDisabled:
    print("No captions available for this video.")

Text Splitting

In [31]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript])

In [33]:
len(chunks)

168

Embedding Genration and Vector Store

In [34]:
embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
vector_store = FAISS.from_documents(chunks, embedding)

Retriever

In [35]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [36]:
retriever.invoke('What is deepmind')

[Document(id='3827dd72-c160-45cc-8af3-4c2234728371', metadata={}, page_content="and how it works this is tough to uh ask you this question because you probably will say it's everything but let's let's try let's try to think to this because you're in a very interesting position where deepmind is the place of some of the most uh brilliant ideas in the history of ai but it's also a place of brilliant engineering so how much of solving intelligence this big goal for deepmind how much of it is science how much is engineering so how much is the algorithms how much is the data how much is the hardware compute infrastructure how much is it the software computer infrastructure yeah um what else is there how much is the human infrastructure and like just the humans interact in certain kinds of ways in all the space of all those ideas how much does maybe like philosophy how much what's the key if um uh if if you were to sort of look back like if we go forward 200 years look back what was the key 

Augmentation

In [37]:
llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash', temperature=0.1)

In [38]:
custom_template = """You are a friendly and helpful assistant specializing in answering questions about a YouTube video transcript.

You must answer ONLY based on the following context. If the context is insufficient, politely say that you don't have enough information from the video transcript.

Chat History:
{chat_history}

Context from Transcript:
{context}

Question: {question}
"""

In [39]:
question          = "is the topic of nuclear fusion discussed in this video? if yes then what was discussed"
retrieved_docs    = retriever.invoke(question)

In [40]:
retrieved_docs

[Document(id='4fe16674-6f4b-4c11-9097-ce649c69ef20', metadata={}, page_content="in this case in fusion we we collaborated with epfl in switzerland the swiss technical institute who are amazing they have a test reactor that they were willing to let us use which you know i double checked with the team we were going to use carefully and safely i was impressed they managed to persuade them to let us use it and um and it's a it's an amazing test reactor they have there and they try all sorts of pretty crazy experiments on it and um the the the what we tend to look at is if we go into a new domain like fusion what are all the bottleneck problems uh like thinking from first principles you know what are all the bottleneck problems that are still stopping fusion working today and then we look at we you know we get a fusion expert to tell us and then we look at those bottlenecks and we look at the ones which ones are amenable to our ai methods today yes right and and and then and would be intere

In [41]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"in this case in fusion we we collaborated with epfl in switzerland the swiss technical institute who are amazing they have a test reactor that they were willing to let us use which you know i double checked with the team we were going to use carefully and safely i was impressed they managed to persuade them to let us use it and um and it's a it's an amazing test reactor they have there and they try all sorts of pretty crazy experiments on it and um the the the what we tend to look at is if we go into a new domain like fusion what are all the bottleneck problems uh like thinking from first principles you know what are all the bottleneck problems that are still stopping fusion working today and then we look at we you know we get a fusion expert to tell us and then we look at those bottlenecks and we look at the ones which ones are amenable to our ai methods today yes right and and and then and would be interesting from a research perspective from our point of view from an ai point of\n\

In [42]:
CUSTOM_PROMPT = PromptTemplate.from_template(custom_template)

Generation

In [43]:
memory = ConversationBufferMemory(
    memory_key="chat_history", 
    return_messages=True
)

Building A Chain

In [44]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [45]:
chat_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever= retriever,
    memory=memory, # <-- This is the crucial line for memory
    combine_docs_chain_kwargs={"prompt": CUSTOM_PROMPT},
    verbose=False
)

In [46]:
question_1 = "Who is Demis Hassabis and what did he invent?"
print(f"User: {question_1}\n")

response_1 = chat_chain.invoke({"question": question_1})

print(f"Assistant: {response_1['answer']}\n")

User: Who is Demis Hassabis and what did he invent?

Assistant: Demis Hassabis is the CEO and co-founder of DeepMind. He is widely considered one of the most brilliant and impactful humans in the history of artificial intelligence, science, and engineering.

DeepMind, the company he co-founded, has published and built incredible artificial intelligence systems, including AlphaGo Zero (which learned to play Go better than any human) and AlphaFold 2 (which solved protein folding).

